In [1]:
import elasticsearch
import re
import yaml
import pandas as pd


from elasticsearch import helpers
# Endereço da plataforma de precedentes: http://precedentes.cloud.fgv.br/search

In [2]:
def get_con_elastic_search():
    from pathlib import Path
    
    my_dir = Path('').absolute()
    serv_config = my_dir/'elastic_search.yaml'
    
    with serv_config.open(mode='r') as f:
        server = yaml.load(f)
        return server
    
def get_con_stf():
    from pathlib import Path
    import sqlalchemy
    
    my_dir = Path('').absolute()
    serv_config = my_dir/'stf_connection.yaml'

    with serv_config.open(mode='r') as f:
        server = yaml.load(f)

    host = server['host']
    database = server['database']
    user = server['user']
    password = server['password']

    url = 'mysql+pymysql://{}:{}@{}/{}'
    url = url.format(user, password, host, database)
    return sqlalchemy.create_engine(url)

In [3]:
con_es = get_con_elastic_search()
con_stf = get_con_stf()

Tarefas:

* Unidade = Processo
* Data - Última decisão disponível
* Eliminar nós que não contém o termo
* Análise de Clusters: Discartar clusters que contém apenas um nó e identificar autoridades dentro de cada cluster.
* Destacar problema com julgamentos em bloco

In [4]:
def busca(termo, con_dict):
    """ Função para executar a busca por um termo na base de decisões
    """
    # parâmetros de conexão com a base
    connection_uri = {'host': 'aplcldrjvpr0017.acad.fgv.br', 'port': 9200}
    # cria a conexão
    connection = elasticsearch.Elasticsearch([connection_uri],
                                       connection_class=elasticsearch.RequestsHttpConnection,
                                       use_ssl=False)
    # criando a consulta
    query = {
        "query" : {
            "query_string": {
                "fields" : ["raw_text"],
                "query": termo
                        }
                }
        }
    
    # retornando o resultado da consulta
    resultado = helpers.scan(client=connection,
                        query=query,
                        index='stf',
                        doc_type='decisoes',
                        request_timeout=120
                        )
    # converte um resutado da busca para uma lista de documentos
    resultado = list(resultado)
    print("Foram encontrados %d results" % len(resultado))
    return resultado

def extract_sig_num(title):
    """ Extrai do título do documento a sigla e o número do processo.
    """
    aux = re.findall("\d+[-|_]([a-zA-Z]+)[-|_](\d+)", title)
    sig = aux[0][0]
    num = aux[0][1]
    return sig, num

def imprime_decisao(decisao):
    sigla, numero = extract_sig_num(decisao['_source']['title'])
    print(u"Processo: " + sigla + " " + numero)

    if decisao['_source'].get('monocratica', False):
        print(u"Decisao monocratica publicada em " + decisao['_source']['date'])
    else:
        print(u"Acordao publicado em " + decisao['_source']['date'])
    
    # imprime apenas os primeiros 100 caracteres
    print(decisao['_source']['raw_text'][:400])

In [5]:
def gerar_citacoes(decisoes):
    arestas = []
    nos = set()
    for decisao in decisoes:
        sigla, numero = extract_sig_num(decisao['_source']['title'])
        node = " ".join([sigla, numero]).upper()
        nos.add(
            (node)
        )
    for decisao in decisoes:
        sigla, numero = extract_sig_num(decisao['_source']['title'])
        node = " ".join([sigla, numero]).upper()
        citacoes = decisao['_source'].get('i_cite', [])
        if len(citacoes) == 0:
            pass
        else:
            for citacao in citacoes:
                if 'STF' not in citacao:
                    pass
                else:
                    kw = str.split(citacao)
                    citado = " ".join(kw[:-2]).upper()
                    
                    if citado in nos:
                        arestas.append((node, citado))
#                     elif "SUM." in citado or "sumula" in citado.lower():
#                         arestas.append((node, citado))
                    else:
                        pass   
    return arestas, nos

def get_soi_data_ultima_decisao(processos, con_stf):
    """Retorna seq_objeto_incidente e data da última decisão do processo
    a partir de um objeto que pode ser convertido para tupla contendo items
    de string no formato classe_processo + ' ' + num_processo"""
    
    query = """
    select processo.seq_objeto_incidente as soi,
            concat(upper(sig_classe_proces), ' ', num_processo) as node,
            andamento_processo.data_andamento
    from STF.processo
    left join
        (select seq_objeto_incidente, data_andamento
         from STF.andamento_processo
         where cod_andamento in 
            (SELECT cod_andamento FROM STF.andamento
             where seq_grupo_andamento in (27, 76, 28, 29)
             )
             ) as andamento_processo
        on processo.seq_objeto_incidente = andamento_processo.seq_objeto_incidente 
    where  concat(upper(sig_classe_proces), ' ', num_processo) in {}
    """.format(tuple(processos))
    
    df = pd.read_sql_query(query, con_stf)
    df.sort_values('data_andamento', inplace=True)
    df.drop_duplicates('soi', keep='last', inplace=True)
    df['node'] = df.node.str.upper()
    df['data_andamento'] = df.data_andamento.dt.strftime('%Y-%m-%d')
    df.columns = ('soi', 'node', 'date')
    return df

def get_cd_assunto_administrativo(df, soi_column, con_stf, desaprop=False, limit_num_ordem=True):
    sois = tuple(df[soi_column].unique())
    if desaprop is True:
        d = "or ramo_do_direito like '%%desapropriacao%%'"
    else:
        d = ""
        
    if limit_num_ordem is True:
        num_ordem = 'num_ordem = 0 and'
    else:
        num_ordem = ''
    query = '''
    SELECT seq_objeto_incidente as soi, cod_assunto
    FROM STF.assunto_processo
    where seq_objeto_incidente in {} and
    {}
    cod_assunto in (
        SELECT cod_assunto
        FROM STF.assunto
        where ramo_do_direito like '%%administrativo%%'
        {});
    '''.format(sois, num_ordem, d)
    df_assunto = pd.read_sql_query(query, con_stf)
    return df.merge(df_assunto, on=soi_column, how='left')

##  Realizando Busca para Moralidade

In [6]:
resultado = busca('"princípio moralidade"~3 OR "moralidade administrativa"~3', con_es)

Foram encontrados 5602 results


In [7]:
arestas, nos = gerar_citacoes(resultado)

In [8]:
len(resultado)

5602

In [9]:
len(nos)

3203

In [10]:
len(arestas)

17081

In [11]:
nos_s = pd.Series(list(nos), name='node').to_frame()

In [12]:
info_nos = get_soi_data_ultima_decisao(nos, con_stf)

In [13]:
info_nos.shape

(3202, 3)

In [14]:
nos_s.query('node not in @info_nos.node')

,node
2606,HC 71820


In [15]:
info_nos.head()

,soi,node,date
4628,1505098,RE 128272,1990-09-26
4629,1505012,RE 128273,1990-09-26
91,1524503,RCL 370,1992-04-09
110,1509143,ADI 402,1993-08-02
460,1610851,PET 999,1995-03-30


Buscando nós em que o código assunto é relativo a Direito Administrativo

In [16]:
info_nos = get_cd_assunto_administrativo(info_nos, 'soi', con_stf, limit_num_ordem=False)

In [17]:
info_nos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3950 entries, 0 to 3949
Data columns (total 4 columns):
soi            3950 non-null int64
node           3950 non-null object
date           3950 non-null object
cod_assunto    2539 non-null float64
dtypes: float64(1), int64(1), object(2)
memory usage: 154.3+ KB


Apenas 1680 dos nós possuem assunto classificado como relacionado a direito administrativo

## Realisando busca para desapropriação

In [18]:
result = busca("desapropriacao~2", con_stf)

Foram encontrados 10761 results


In [19]:
l=[]
for x in result:
    l.append(" ".join(extract_sig_num(x['_source']['title'])))

In [20]:
edges, nodes = gerar_citacoes(result)

In [21]:
nodes_s = pd.Series(list(nodes), name='node').to_frame()

In [22]:
len(nodes)

6707

In [23]:
len(edges)

15428

In [24]:
info_nodes = get_soi_data_ultima_decisao(nodes, con_stf)

In [25]:
info_nodes.shape

(6706, 3)

In [27]:
info_nodes = get_cd_assunto_administrativo(info_nodes, 'soi', con_stf=con_stf,
                                           desaprop=True, limit_num_ordem=False)

In [28]:
info_nodes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7872 entries, 0 to 7871
Data columns (total 4 columns):
soi            7872 non-null int64
node           7872 non-null object
date           7872 non-null object
cod_assunto    5829 non-null float64
dtypes: float64(1), int64(1), object(2)
memory usage: 307.5+ KB


Dos 6706 processos, 4305 possuem assunto codificado como relativo a Direito Administrativo

In [29]:
nodes_s.query('node not in @info_nodes.node')

,node
4765,ARE 739313


## Gerando Grafos

In [30]:
import networkx as nx
from datetime import datetime
        
def add_nodes_attributes(graph, att_df):
    for col in att_df.columns:
        if col != 'node':
            nx.set_node_attributes(
                graph,
                pd.Series(att_df[col].values,
                          index = att_df.node).to_dict(),
                name=col
            )
    return graph

def create_graph(nodes, edges, attrib=False, att_df=None,
                 export=False, fn=None, filter_cd_assunto=False):
    graph = nx.DiGraph()
    
    graph.add_nodes_from(nodes)
    graph.add_edges_from(edges)
    
    if attrib:
        graph = add_nodes_attributes(graph, att_df)
    
    if filter_cd_assunto is True:
        nodes_with_assunto = set(att_df.query('cod_assunto == cod_assunto')['node'])
        graph = graph.subgraph(nodes_with_assunto)
    
    if export:
        nx.write_gexf(graph, path=fn)
    return graph

Grafo de moralidade

In [31]:
m = create_graph(nos, arestas,
                 att_df=info_nos, attrib=True,
                 export=True, fn='grafo_moralidade_pre-gephi.gexf',
                 filter_cd_assunto=True)

In [32]:
len(m.nodes)

1791

In [33]:
len(m.edges)

730

Grafo desapropriação

In [34]:
desaprop = create_graph(nodes, edges,
                        att_df=info_nodes, attrib=True,
                        export=True, fn='grafo_desapropriacao_pre-gephi.gexf',
                        filter_cd_assunto=True)

In [35]:
len(desaprop.nodes)

4663

In [36]:
len(desaprop.edges)

5903

Checando self loops

In [37]:
m.nodes_with_selfloops()

<generator object nodes_with_selfloops.<locals>.<genexpr> at 0x7f849fa67d00>

Chegando ciclos

In [38]:
# for c in nx.simple_cycles(m):
#     print(c)

AttributeError: 'OutEdgeView' object has no attribute 'graph'

In [ ]:
# for c in nx.simple_cycles(desaprop):
#     print(c)